In [151]:
import pandas as pd
import numpy as np
import time

In [3]:
trails = pd.read_csv('PACR_DATA_2018-01-08_1413.csv')

In [4]:
# take a look at the damage
trails


,lab_id,bike_id,tm_a,tm_ae,tm_b,tm_be,at,pae,bt,be
0,1,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,44,8,23.02,0,68.01,0,23 seconds,0 error,44 seconds,0 error
2,45,41,28,0,39,0,NaN,NaN,NaN,NaN
3,58,25,28 seconds,0,41 seconds,0,21.61,0,48.51,0
4,68,12,42,0,62,0,26 seconds,0 error,61 seconds,0 error
5,87,7,29 seconds,0,60 seconds,0,27 seconds,0,63 seconds,0
6,92,9,41 seconds,1,82 seconds,0,39.51,1,72.12,0
7,157,51,28 seconds,0,54 seconds,0,23.93,0,45.08,4
8,182,31,37 seconds,0,75 seconds,0,36,0,63,0
9,200,13,43 seconds,0,58 seconds,0,40.3,1,64.93,0


## Several items we need to consider:
- How should the data be represented? earlier it was reported as integers, but later decimals were introduced.
    - just going to go with float representation for now to keep the detailed data detailed
- Removal of strings (e.g. the 'seconds' and 'errors' strings)
    - going to make a function to format strings to floats
- Transfer the 1:00 representations of time to seconds
    - going to make a function to format time to seconds
- How should NANs and 0s be represented
    - As they are since they have differential meanings, (e.g. NANs mean did not complete, whereas zeros mean there were not any errors) (hopefully there are no cases where it took 0 seconds to complete the task)

In [101]:
# transfers 1:20(.22) to 80(.22)
def minutes2seconds(m):
    # if there are milliseconds
    if m.find('.') > 0:
        # treat ms separately and add them in at the end
        tim, ms = m.split('.',1)
    else:
        # ignore ms, which will treated as 0
        tim = m
        ms = ''
    if ms != '':
        # convert milliseconds to their appropiate format
        ms = float(ms) / 100
    else:
        # treat milliseconds as zero (if it doesn't exist)
        ms = 0
    # make a time object to parse minutes and seconds
    tt=time.strptime(tim,"%M:%S")
    # add all the components up.
    return tt.tm_min * 60 + tt.tm_sec + ms

def format_series(s):
    # if the datatype is a string and its a time (e.g. 1:20)
    if type(s) is str and s.find(':') > 0:
        out = float(minutes2seconds(s))
    # if the datatype is just a string (e.g. '54 seconds')
    elif type(s) is str:
        out = float(s.rstrip('[secondserrors ]'))
    # else the datatype is already a number
    else:
        out = float(s)
    return out

# main function
def remove_str(series):
    return series.map(format_series)
    

In [104]:
# apply the function to all columns
trails_fmt = trails.apply(remove_str)

In [106]:
# lets see how much better the data look
trails_fmt

,lab_id,bike_id,tm_a,tm_ae,tm_b,tm_be,at,pae,bt,be
0,1.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,44.0,8.0,23.02,0.0,68.01,0.0,23.00,0.0,44.00,0.0
2,45.0,41.0,28.00,0.0,39.00,0.0,NaN,NaN,NaN,NaN
3,58.0,25.0,28.00,0.0,41.00,0.0,21.61,0.0,48.51,0.0
4,68.0,12.0,42.00,0.0,62.00,0.0,26.00,0.0,61.00,0.0
5,87.0,7.0,29.00,0.0,60.00,0.0,27.00,0.0,63.00,0.0
6,92.0,9.0,41.00,1.0,82.00,0.0,39.51,1.0,72.12,0.0
7,157.0,51.0,28.00,0.0,54.00,0.0,23.93,0.0,45.08,4.0
8,182.0,31.0,37.00,0.0,75.00,0.0,36.00,0.0,63.00,0.0
9,200.0,13.0,43.00,0.0,58.00,0.0,40.30,1.0,64.93,0.0


## Give Informative Column Names
To tidy up some more, we will change the column names to more informative/accurate names

In [114]:
col_names = {'lab_id': 'lab_id', 
             'bike_id': 'pacr_id', 
             'tm_a': 'pre_time_a', 
             'tm_ae': 'pre_err_a', 
             'tm_b': 'pre_time_b', 
             'tm_be': 'pre_err_b', 
             'at': 'post_time_a', 
             'pae': 'post_err_a', 
             'bt': 'post_time_b', 
             'be': 'post_err_b'}

trails_fmt.rename(columns=col_names, inplace=True)

## Overzealous Floats
I was a bit overzealous with converting to floats, so I should change lab_id, pacr_id, and the err columns to integers

EDIT: [pandas represents NaNs as floats, can't change them](https://pandas.pydata.org/pandas-docs/stable/gotchas.html#nan-integer-na-values-and-na-type-promotions). 

So I will only be changing lab_id and pacr_id as integers


In [159]:
trails_fmt[['lab_id', 'pacr_id']] = trails_fmt[['lab_id','pacr_id']].astype(int)

In [161]:
# lets see the tidier dataset
trails_fmt

,lab_id,pacr_id,pre_time_a,pre_err_a,pre_time_b,pre_err_b,post_time_a,post_err_a,post_time_b,post_err_b
0,1,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,44,8,23.02,0.0,68.01,0.0,23.00,0.0,44.00,0.0
2,45,41,28.00,0.0,39.00,0.0,NaN,NaN,NaN,NaN
3,58,25,28.00,0.0,41.00,0.0,21.61,0.0,48.51,0.0
4,68,12,42.00,0.0,62.00,0.0,26.00,0.0,61.00,0.0
5,87,7,29.00,0.0,60.00,0.0,27.00,0.0,63.00,0.0
6,92,9,41.00,1.0,82.00,0.0,39.51,1.0,72.12,0.0
7,157,51,28.00,0.0,54.00,0.0,23.93,0.0,45.08,4.0
8,182,31,37.00,0.0,75.00,0.0,36.00,0.0,63.00,0.0
9,200,13,43.00,0.0,58.00,0.0,40.30,1.0,64.93,0.0


## Calculating Time Cost
Finally, we need to calculate time cost for pre and post

In [166]:
trails_fmt['pre_timecost'] = trails_fmt['pre_time_b'] - trails_fmt['pre_time_a']
trails_fmt['post_timecost'] = trails_fmt['post_time_b'] - trails_fmt['post_time_a']

In [170]:
# Finished! (just sorting by pacr_id)
trails_fmt.sort_values(by=['pacr_id'], inplace=True)

In [177]:
# eliminating subjects who didn't have a pre_timecost
trails_fmt = trails_fmt[np.isfinite(trails_fmt['pre_timecost'])]

In [184]:
# mistype in REDCAP, changing 3.25 to 30.25 (what's intended)
trails_fmt['post_time_a'][92] = 30.25

/home/jdkent/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [188]:
# write it to file.
trails_fmt.to_csv('./trails_fmt.tsv', sep='\t')